<a href="https://colab.research.google.com/github/ge43jef/GEEHYDRO/blob/block3/regression_linear_polynomial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scikit-Learn Regression
There is an open-source, commercially usable machine learning toolkit called scikit-learn. This toolkit contains implementations of many of the algorithms that you will work with in this course.

## Goals
In this lab you will:
- Utilize  scikit-learn to implement linear regression
- Utilize  scikit-learn to implement polynomial regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import learning_curve

## A simple linear regression sample
Scikit-learn has the [linear regression model](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) which implements a closed-form linear regression.

Let's use the data from the early labs - a basin with evaporation rate of 5 mm/d under the average temperature 15.0 degree and a basin with evaporation rate of 13 mm/d under the average temperature 25.5 degree.

| evaporation rate(mm/d)     | average temperature(degree) |
| -------------------| ------------------------ |
| 5.0               | 15.0                      |
| 13.0               | 25.5                      |

### Load the data set

In [ ]:
X_train = np.array([5.0, 15.0])   #features
y_train = np.array([13.0, 25.5])   #target value

### Create and fit the model
The code below performs regression using scikit-learn.
The first step creates a regression object.
The second step utilizes one of the methods associated with the object, `fit`. This performs regression, fitting the parameters to the input data. The toolkit expects a two-dimensional X matrix.

In [ ]:
linear_model = LinearRegression()
#X must be a 2-D Matrix
linear_model.fit(X_train.reshape(-1, 1), y_train)

### View Parameters
The $\mathbf{w}$ and $\mathbf{b}$ parameters are referred to as 'coefficients' and 'intercept' in scikit-learn.

In [ ]:
b = linear_model.intercept_
w = linear_model.coef_
print(f"w = {w:}, b = {b:0.2f}")
print(f"'manual' prediction: f_wb = wx+b : {1200*w + b}")

### Make Predictions

Calling the `predict` function generates predictions.

In [ ]:
y_pred = linear_model.predict(X_train.reshape(-1, 1))

print("Prediction on training set:", y_pred)

X_test = np.array([[12]])
print(f"Prediction for evaporation rate under 12 degree: {linear_model.predict(X_test)[0]:0.2f}mm/d")

In [ ]:
# plot the data
# Plot the data points
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line, = ax.plot(X_train, y_pred, c='b',label='Our Prediction')
plt.scatter(X_train, y_train, marker='x', c='r',label='True Value')
# Set the y-axis label
plt.ylabel('temperature (degree)')
# Set the x-axis label
plt.xlabel('evaporation rate (mm/d)')
plt.legend()
plt.show()

## An application case in hydrology
The [FLUXNET2015](https://fluxnet.org/) Dataset includes data collected at sites from multiple regional flux networks, it provides ecosystem-scale data on CO2, water, and energy exchange between the biosphere and the atmosphere, and other meteorological and biological measurements.

In this lab, you will use six variables: incoming shortwave radiation(W m-2), incoming longwave radiation(W m-2), temperature(degree C), pressure(kPa), wind speed(m/s), vapor pressure deficit(hPa) to predict the **latent heat flux(W m-2)**, which is the energy form of the evaporation.

| variable     | notation |
| -------------------| ------------------------ |
| incoming shortwave radiation               | sw                      |
| incoming longwave radiation              | lw                      |
| temperature               | tmp                      |
| pressure               | pre                      |
| wind speed               | u10                      |
| vapor pressure deficit               | vpd                     |
| latent heat flux               | lh                      |

In [ ]:
data = pd.read_csv('FLX_US-Ne1_FLUXNET2015_SUBSET_DD_2001-2013_1-4.csv' , delimiter=",", skipinitialspace=True,  parse_dates=True)

meteo = pd.DataFrame(
            {"sw": data.SW_IN_F, "lw": data.LW_IN_F, "tmp": data.TA_F,
             "pre": data.PA_F, "u10": data.WS_F,  "vpd": data.VPD_F , "lh": data.LE_CORR})

data_all = np.array(meteo)
X = data_all[ : , 0:6]
y = data_all[ : , 6]

### Read data and split the data into train and test sample without KFold(cross validation)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
linear_model = LinearRegression()
linear_model.fit(X_train , y_train)
y_pred = linear_model.predict(X_test)

# plot the data
# Plot the data points
fig = plt.figure()
fig,ax=plt.subplots(2, 1, figsize=(6, 12), sharey=True)
ax[0].plot( y_test, marker='x', c='r',label='True Value')
ax[0].plot( y_pred, c='b',label='Our Prediction')
ax[0].set(xlabel="time (day)", ylabel="evaporation rate (mm/d)")
ax[1].scatter( y_test , y_pred, c='b')
z = np.polyfit(y_test , y_pred, 1)
y_hat = np.poly1d(z)(y_pred)
plt.plot(y_pred, y_hat, "r--", lw=2)
text = f"$y={z[0]:0.3f}\;x{z[1]:+0.3f}$\n$R^2 = {r2_score(y_test, y_hat):0.3f}$\n" \
                   f"$RMSE = {mean_squared_error(y_test, y_hat, squared=False):0.3f} $ "
plt.gca().text(0.05, 0.95, text, transform=plt.gca().transAxes,
                           fontsize=14, verticalalignment='top')
plt.ylabel('Predict Value')
# Set the x-axis label
plt.xlabel('True Value')
ax[0].legend()
print("coefficient of determination R^2 =",linear_model.score(X_test , y_test.reshape(-1, 1)))
plt.show()

### Read data and split the data into train and test sample with KFold

In [ ]:
kf = KFold(n_splits=5)
for train, test in kf.split(X):
    x_train = X[train]

In [ ]:
for train, test in kf.split(X):
    X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
    linear_model = LinearRegression()
    linear_model.fit(X_train , y_train)
    y_pred = linear_model.predict(X_test)

In [ ]:
y_pred = linear_model.predict(X_test)
# plot the data
# Plot the data points
fig = plt.figure()
fig,ax=plt.subplots(2, 1, figsize=(6, 12), sharey=True)
ax[0].plot( y_test, marker='x', c='r',label='True Value')
ax[0].plot( y_pred, c='b',label='Our Prediction')
ax[0].set(xlabel="time (day)", ylabel="evaporation rate (mm/d)")
ax[1].scatter( y_test , y_pred, c='b')
z = np.polyfit(y_test , y_pred, 1)
y_hat = np.poly1d(z)(y_pred)
plt.plot(y_pred, y_hat, "r--", lw=2)
text = f"$y={z[0]:0.3f}\;x{z[1]:+0.3f}$\n$R^2 = {r2_score(y_test, y_hat):0.3f}$\n" \
                   f"$RMSE = {mean_squared_error(y_test, y_hat, squared=False):0.3f} $ "
plt.gca().text(0.05, 0.95, text, transform=plt.gca().transAxes,
                           fontsize=14, verticalalignment='top')
plt.ylabel('Predict Value')
# Set the x-axis label
plt.xlabel('True Value')
ax[0].legend()
print("coefficient of determination R^2 =",linear_model.score(X_test , y_test.reshape(-1, 1)))
plt.show()

In [ ]:
# plot scatter: x vs y

X_features = ['sw','lw','tmp','pre','u10','vpd']
fig,ax=plt.subplots(2, 3, figsize=(12, 12), sharey=True)
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.2,
                    hspace=0.2)
for i in range(2):
    for j in range(3):
        ax[i , j].scatter(X_train[:,3 * i+j],y_train)
        ax[i , j].set_xlabel(X_features[3 *i+j])

ax[0 , 0].set_ylabel("lh")
plt.show()

In [ ]:
# plot scatter: x^2 vs y

X_features = ['sw','lw','tmp','pre','u10','vpd']
fig,ax=plt.subplots(2, 3, figsize=(12, 12), sharey=True)
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.2,
                    hspace=0.2)
for i in range(2):
    for j in range(3):
        ax[i , j].scatter(X_train[:,3 * i+j] **2 ,y_train)
        ax[i , j].set_xlabel(X_features[3 *i+j])

ax[0 , 0].set_ylabel("lh")
plt.show()

In [ ]:
# plot scatter: x^3 vs y

X_features = ['sw','lw','tmp','pre','u10','vpd']
fig,ax=plt.subplots(2, 3, figsize=(12, 12), sharey=True)
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.2,
                    hspace=0.2)
for i in range(2):
    for j in range(3):
        ax[i , j].scatter(X_train[:,3 * i+j] **3 ,y_train)
        ax[i , j].set_xlabel(X_features[3 *i+j])

ax[0 , 0].set_ylabel("lh")
plt.show()

In [ ]:
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X[: , 0:3])
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, shuffle=False)

linear_model = LinearRegression()
linear_model.fit(X_train , y_train)
y_pred = linear_model.predict(X_test)

fig = plt.figure()
fig,ax=plt.subplots(2, 1, figsize=(6, 12), sharey=True)
ax[0].plot( y_test, marker='x', c='r',label='True Value')
ax[0].plot( y_pred, c='b',label='Our Prediction')
ax[0].set(xlabel="time (day)", ylabel="evaporation rate (mm/d)")
ax[1].scatter( y_test , y_pred, c='b')
z = np.polyfit(y_test , y_pred, 1)
y_hat = np.poly1d(z)(y_pred)
plt.plot(y_pred, y_hat, "r--", lw=2)
text = f"$y={z[0]:0.3f}\;x{z[1]:+0.3f}$\n$R^2 = {r2_score(y_test, y_hat):0.3f}$\n" \
                   f"$RMSE = {mean_squared_error(y_test, y_hat, squared=False):0.3f} $ "
plt.gca().text(0.05, 0.95, text, transform=plt.gca().transAxes,
                           fontsize=14, verticalalignment='top')
plt.ylabel('Predict Value')
# Set the x-axis label
plt.xlabel('True Value')
ax[0].legend()
plt.show()

In [ ]:
# Polynomial regression with k-fold
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X[: , 0:3])

for train, test in kf.split(X):
    X_train, X_test, y_train, y_test = X_poly[train], X_poly[test], y[train], y[test]
    linear_model = LinearRegression()
    linear_model.fit(X_train , y_train)
    y_pred = linear_model.predict(X_test)

fig = plt.figure()
fig,ax=plt.subplots(2, 1, figsize=(6, 12), sharey=True)
ax[0].plot( y_test, marker='x', c='r',label='True Value')
ax[0].plot( y_pred, c='b',label='Our Prediction')
ax[0].set(xlabel="time (day)", ylabel="evaporation rate (mm/d)")
ax[1].scatter( y_test , y_pred, c='b')
z = np.polyfit(y_test , y_pred, 1)
y_hat = np.poly1d(z)(y_pred)
plt.plot(y_pred, y_hat, "r--", lw=2)
text = f"$y={z[0]:0.3f}\;x{z[1]:+0.3f}$\n$R^2 = {r2_score(y_test, y_hat):0.3f}$\n" \
                   f"$RMSE = {mean_squared_error(y_test, y_hat, squared=False):0.3f} $ "
plt.gca().text(0.05, 0.95, text, transform=plt.gca().transAxes,
                           fontsize=14, verticalalignment='top')
plt.ylabel('Predict Value')
# Set the x-axis label
plt.xlabel('True Value')
ax[0].legend()
# print("coefficient of determination R^2 =",linear_model.score(X_test , y_test.reshape(-1, 1)))
plt.show()

### Please use sklearn function to check the best parameters, and other score metrics

## Another training method
A training method with [learning curve](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.learning_curve.html), please use this method with other models.
Below is an example with polynomial regression model.

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(estimator=linear_model, X=X_train, y=y_train,
                                                       cv=5, train_sizes=np.linspace(0.1, 1.0, 10))
#
# Calculate training and test mean and std
#
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean, color='green', marker='+', markersize=5, linestyle='--', label='Validation Accuracy')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.title('Learning Curve')
plt.xlabel('Training Data Size')
plt.ylabel('Model accuracy')
plt.grid()
plt.legend(loc='lower right')
plt.show()